# Main Notebook
Notebook contains training for model and eventually trained model. It also includes a way to test new images on the model.

In [ ]:
import os

pos_folder = r'path\to\folder'
neg_folder = r'path\to\folder'

# List to store preprocessed images and labels
preprocessed_images = []
labels = []

# Positive class (golf ball present)
for filename in os.listdir(pos_folder):
    if filename.lower().endswith(".jpg"):
        image_path = os.path.join(pos_folder, filename)
        preprocessed_image = preprocess_image(image_path)
        preprocessed_images.append(preprocessed_image)
        labels.append(1)

# Negative class (no golf ball)
for filename in os.listdir(neg_folder):
    if filename.lower().endswith(".jpg"):
        image_path = os.path.join(neg_folder, filename)
        preprocessed_image = preprocess_image(image_path)
        preprocessed_images.append(preprocessed_image)
        labels.append(0)

# Convert lists to NumPy arrays for further processing
X = np.array(preprocessed_images)
y = np.array(labels)

# Verify shapes (should match number of images processed)
print("X shape:", X.shape)
print("y shape:", y.shape)

In [113]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split

# Load CSV with filenames and labels
labels_df = pd.read_csv('labels.csv')  # Adjust file path as necessary

# Path to your image directory
image_dir = r'path\to\folder'

# Function to load and preprocess images
def load_and_preprocess_images(image_dir, labels_df):
    images = []
    labels = []
    for index, row in labels_df.iterrows():
        filename = row['filename']  # Assuming CSV has a 'filename' column
        label = row['label']  # Assuming CSV has a 'label' column (0 or 1)
        img = load_img(image_dir + '\\' + filename, target_size=(224, 224))  # Adjust target_size as needed
        img = img_to_array(img)
        img = preprocess_input(img)
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_and_preprocess_images(image_dir, labels_df)

# Split data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [114]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load pre-trained InceptionV3 model without top layers
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [115]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add your custom classifier layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
# Combine base model and custom layers into a new model
model = Model(inputs=base_model.input, outputs=predictions)

In [126]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/10
2/2 [==============================] - 10s 2s/step - loss: 0.7209 - accuracy: 0.6607 - val_loss: 0.6892 - val_accuracy: 0.5714
Epoch 2/10
2/2 [==============================] - 1s 535ms/step - loss: 0.6230 - accuracy: 0.6607 - val_loss: 0.6645 - val_accuracy: 0.4286
Epoch 3/10
2/2 [==============================] - 1s 503ms/step - loss: 0.6779 - accuracy: 0.4821 - val_loss: 0.6034 - val_accuracy: 0.7143
Epoch 4/10
2/2 [==============================] - 1s 503ms/step - loss: 0.6220 - accuracy: 0.7143 - val_loss: 0.6148 - val_accuracy: 0.5714
Epoch 5/10
2/2 [==============================] - 1s 536ms/step - loss: 0.6116 - accuracy: 0.6607 - val_loss: 0.5899 - val_accuracy: 0.7143
Epoch 6/10
2/2 [==============================] - 1s 511ms/step - loss: 0.5999 - accuracy: 0.6964 - val_loss: 0.5485 - val_accuracy: 0.7143
Epoch 7/10
2/2 [==============================] - 1s 495ms/step - loss: 0.5798 - accuracy: 0.7321 - val_loss: 0.5145 - val_accuracy: 0.8571
Epoch 8/10
2/2 [======

# Test Model
Code to process a new image test model.

In [1]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Example of loading and preprocessing a single new image
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)  # Add batch dimension
    return img_array

# Example usage:
new_image_path = r'path\to\folder'
target_size = (224, 224)  # Adjust according to your model's input size

preprocessed_image = preprocess_image(new_image_path, target_size)

# Assuming `model` is your trained Keras model
predictions = model.predict(preprocessed_image)

# Assuming binary classification (0 or 1)
predicted_class = predictions[0][0]  # Accessing the first prediction (single image, single class)

print(predicted_class)

if predicted_class >= 0.5:
    print("Predicted: Golf ball present")
else:
    print("Predicted: No golf ball")

NameError: name 'model' is not defined